# Dendritic YOLOv8 - Object Detection with Dendritic Optimization

This notebook demonstrates integrating PerforatedAI's dendritic optimization into YOLOv8n.

**Important**: This notebook produces the correct PAI graph format as shown in `Dendrite Recommendations.pdf`:
- Green line: Training scores
- Orange line: Validation scores  
- Blue/Red lines: What would have happened without dendrites
- Vertical bars: Epochs where dendrites were added

## Setup
1. **Runtime -> Change runtime type -> GPU (T4)**
2. **Run all cells in order**

In [ ]:
# Cell 1: Install dependencies
!pip install -q ultralytics
!pip install -q perforatedai==3.0.7

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Cell 2: Imports
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
import json
from ultralytics import YOLO

# PerforatedAI imports
from perforatedai import globals_perforatedai as GPA
from perforatedai import utils_perforatedai as UPA

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

In [ ]:
# Cell 3: Helper functions
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
# Cell 4: Load model and initialize PerforatedAI
print("Loading YOLOv8n...")
yolo = YOLO('yolov8n.pt')
model = yolo.model

baseline_params = count_params(model)
print(f"Baseline parameters: {baseline_params:,}")

# Configure PerforatedAI
GPA.pc.set_testing_dendrite_capacity(False)
GPA.pc.set_verbose(True)
GPA.pc.set_dendrite_update_mode(True)
GPA.pc.set_max_dendrites(5)  # Allow up to 5 dendrite addition cycles

# Create output directory
os.makedirs('PAI', exist_ok=True)

# Initialize PAI - this enables dendrite optimization
model = UPA.initialize_pai(
    model,
    save_name='PAI',  # Output will be PAI/PAI.png
    maximizing_score=True,  # We maximize mAP
    making_graphs=True
)

model = model.to(device)
yolo.model = model
print("PerforatedAI initialized!")

In [ ]:
# Cell 5: Setup optimizer through PAI tracker
GPA.pai_tracker.set_optimizer(optim.Adam)
GPA.pai_tracker.set_scheduler(ReduceLROnPlateau)

lr = 0.001
optimArgs = {'params': model.parameters(), 'lr': lr}
schedArgs = {'mode': 'max', 'patience': 5}
optimizer, scheduler = GPA.pai_tracker.setup_optimizer(model, optimArgs, schedArgs)

print("Optimizer configured through PAI tracker")

In [ ]:
# Cell 6: Training loop with PROPER PAI integration
#
# KEY for correct graph output:
# 1. add_extra_score(train_score, 'train') -> Creates GREEN line
# 2. add_validation_score(val_score, model) -> Creates ORANGE line & triggers dendrites
# 3. Continue until training_complete is True
# 4. Reset optimizer when model is restructured

MAX_EPOCHS = 100
DATA = 'coco128.yaml'
BATCH = 16
IMGSZ = 640

print(f"Starting training with PerforatedAI integration...")
print(f"Baseline parameters: {baseline_params:,}")
print("="*60)

epoch = 0
training_complete = False
history = {'train': [], 'val': [], 'params': []}

while not training_complete and epoch < MAX_EPOCHS:
    print(f"\nEpoch {epoch + 1}/{MAX_EPOCHS}")
    
    # Train one epoch
    yolo.model = model
    results = yolo.train(
        data=DATA,
        epochs=1,
        imgsz=IMGSZ,
        batch=BATCH,
        device=device,
        exist_ok=True,
        verbose=False,
        project='runs/train',
        name='dendritic'
    )
    model = yolo.model
    
    # Get training score
    train_map50 = float(results.results_dict.get('metrics/mAP50(B)', 0))
    
    # CRITICAL: Add TRAINING score (creates green line)
    GPA.pai_tracker.add_extra_score(train_map50 * 100, 'train')
    
    # Validate
    val_metrics = yolo.val(verbose=False)
    val_map50 = float(val_metrics.box.map50)
    val_map50_95 = float(val_metrics.box.map)
    
    # CRITICAL: Add VALIDATION score (creates orange line, may add dendrites)
    model, restructured, training_complete = GPA.pai_tracker.add_validation_score(
        val_map50_95 * 100, model
    )
    model = model.to(device)
    yolo.model = model
    
    # Log
    current_params = count_params(model)
    history['train'].append(train_map50)
    history['val'].append(val_map50_95)
    history['params'].append(current_params)
    
    print(f"  Train mAP@0.5: {train_map50:.4f}")
    print(f"  Val mAP@0.5:   {val_map50:.4f}")
    print(f"  Val mAP@0.5:0.95: {val_map50_95:.4f}")
    print(f"  Parameters: {current_params:,}")
    
    # If restructured, reset optimizer
    if restructured:
        print("\n>>> DENDRITES ADDED! Model restructured <<<")
        optimArgs['params'] = model.parameters()
        optimizer, scheduler = GPA.pai_tracker.setup_optimizer(model, optimArgs, schedArgs)
    
    if training_complete:
        print("\n" + "="*60)
        print("TRAINING COMPLETE!")
        print("="*60)
    
    epoch += 1

print(f"\nFinished after {epoch} epochs")

In [ ]:
# Cell 7: Save graphs and results
print("Saving PAI graphs...")
try:
    GPA.pai_tracker.save_graphs()
    print("Graphs saved to PAI/PAI.png")
except Exception as e:
    print(f"Note: {e}")

# Final results
final_params = count_params(model)
final_val = yolo.val(verbose=False)

print("\n" + "="*60)
print("FINAL RESULTS")
print("="*60)
print(f"Baseline parameters:  {baseline_params:,}")
print(f"Final parameters:     {final_params:,}")
print(f"Parameter change:     {((final_params - baseline_params) / baseline_params) * 100:+.1f}%")
print(f"Final mAP@0.5:        {final_val.box.map50:.4f}")
print(f"Final mAP@0.5:0.95:   {final_val.box.map:.4f}")
print("="*60)

# Save results
results = {
    'baseline_params': baseline_params,
    'final_params': final_params,
    'final_mAP50': float(final_val.box.map50),
    'final_mAP50_95': float(final_val.box.map),
    'epochs_trained': epoch,
    'history': history
}
with open('PAI/results.json', 'w') as f:
    json.dump(results, f, indent=2)
print("\nResults saved to PAI/results.json")

In [ ]:
# Cell 8: Display the PAI graph
from IPython.display import Image, display

if os.path.exists('PAI/PAI.png'):
    print("PAI Output Graph (matches Dendrite Recommendations format):")
    print()
    display(Image('PAI/PAI.png'))
    print("\nGraph interpretation:")
    print("- Green line: Training scores")
    print("- Orange line: Validation scores")
    print("- Vertical bars: Dendrite addition epochs")
    print("- Blue/Red lines: What would have happened without dendrites")
else:
    print("Graph not found. Ensure training completed.")

In [ ]:
# Cell 9: Download results
try:
    from google.colab import files
    
    if os.path.exists('PAI/PAI.png'):
        files.download('PAI/PAI.png')
    if os.path.exists('PAI/results.json'):
        files.download('PAI/results.json')
    
    # Download CSVs
    for f in os.listdir('PAI'):
        if f.endswith('.csv'):
            files.download(f'PAI/{f}')
    
    print("Files downloaded!")
except:
    print("Files are in PAI/ directory")